In [ ]:
!pip install anvil-uplink

In [ ]:
# Step 2: Import necessary libraries
import numpy as np
import pickle
import anvil.server
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import requests
from bs4 import BeautifulSoup

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import Counter
import string
from nltk.stem import PorterStemmer
import re
import anvil.server

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

In [ ]:
# Step 3: Connect to Anvil app using uplink key
anvil.server.connect("xxx")

In [ ]:
# Step 4: Load the model and tokenizer
model = load_model('/content/drive/MyDrive/FYP ITEMS/news_cnn_model.keras')
with open('/content/drive/MyDrive/FYP ITEMS/tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:

import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk

# Download NLTK stopwords if not already available
nltk.download('stopwords')

# Initialize stopwords set and stemmer
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()

# Define the text preprocessing function
def wordopt(text):
    # Lowercase the text
    text = text.lower()
    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    # Remove punctuation and digits
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d', '', text)
    # Remove newlines
    text = re.sub(r'\n', ' ', text)
    # Split text into words
    words = text.split()
    # Remove stopwords and apply stemming
    words = [ps.stem(word) for word in words if word not in stop_words]
    # Join words back into a single string
    return ' '.join(words)

# Updated preprocessing function
def preprocess_input(news, tokenizer, max_len=500):
    # Apply text preprocessing (wordopt) before tokenizing
    news = wordopt(news)
    news_seq = tokenizer.texts_to_sequences([news])
    news_pad = pad_sequences(news_seq, maxlen=max_len)
    return news_pad

max_len=500

# Output label function
def output_label(n):
    if n == 0:
        return "It is Fake News"
    elif n == 1:
        return "It is Genuine News"


In [ ]:
#  Step 6: Define a function for manual testing
@anvil.server.callable
def manual_testing(news):
    testing_news = [news]
    testing_news_seq = tokenizer.texts_to_sequences(testing_news)
    testing_news_pad = pad_sequences(testing_news_seq, maxlen=max_len)
    pred_cnn = model.predict(testing_news_pad)
    return output_label(int(np.round(pred_cnn[0])))
